In [41]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
import pandas as pd

# Load the Excel dataset (make sure it's in the same folder)
df = pd.read_excel("C:/Users/Admin/Downloads/police project 1/traffic_stops.xlsx")

# Drop unnecessary columns (we only need cleaned ones)
df.drop(['driver_age_raw', 'violation_raw'], axis=1, inplace=True)

# Combine date and time into one column
df['stop_datetime'] = pd.to_datetime(df['stop_date'].astype(str) + ' ' + df['stop_time'].astype(str))

# Drop old date and time columns
df.drop(['stop_date', 'stop_time'], axis=1, inplace=True)

# Move 'stop_datetime' to the front
cols = ['stop_datetime'] + [col for col in df.columns if col != 'stop_datetime']
df = df[cols]

# Show the first 5 rows
df.head()


,stop_datetime,country_name,driver_gender,driver_age,driver_race,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01 00:00:00,Canada,M,19,Asian,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01 00:01:00,India,M,58,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01 00:02:00,USA,M,76,Black,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01 00:03:00,Canada,M,76,Black,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01 00:04:00,Canada,M,75,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [44]:
import mysql.connector

# Connect to XAMPP MySQL (root user, no password)
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""  # Leave empty if you didn't set one
)

# Create SQL cursor
cursor = mydb.cursor()

# Create new database
cursor.execute("CREATE DATABASE IF NOT EXISTS securecheck_db")
 
# Use the new database
cursor.execute("USE securecheck_db")

print("Connected to MySQL & using database: securecheck_db")


Connected to MySQL & using database: securecheck_db


In [45]:
pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
import mysql.connector
import pandas as pd

# Step 1: Connect to MySQL (XAMPP)
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",              # Leave blank if no password is set in XAMPP
    database="securecheck"    # Make sure you've created this in phpMyAdmin
)

mycursor = mydb.cursor()

# Step 2: Create the table if it doesn't exist
mycursor.execute("""
CREATE TABLE IF NOT EXISTS traffic_logs (
    stop_datetime DATETIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    is_arrested BOOLEAN,
    stop_duration VARCHAR(50),
    drugs_related_stop BOOLEAN
)
""")

# Step 3: Define only required columns from DataFrame
columns = ['stop_datetime', 'country_name', 'driver_gender', 'driver_age', 'driver_race',
           'violation', 'search_conducted', 'search_type', 'stop_outcome',
           'is_arrested', 'stop_duration', 'drugs_related_stop']

# Step 4: Insert each row safely (replace NaN with None)
for index, row in df[columns].iterrows():
    values = [None if pd.isna(x) else x for x in row]
    mycursor.execute("""
        INSERT INTO traffic_logs (
            stop_datetime, country_name, driver_gender, driver_age, driver_race,
            violation, search_conducted, search_type, stop_outcome,
            is_arrested, stop_duration, drugs_related_stop
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, values)

# Step 5: Save and close
mydb.commit()
mycursor.close()
mydb.close()

print(" All data inserted into MySQL successfully!")


 All data inserted into MySQL successfully!


In [48]:
import mysql.connector
import pandas as pd

# Connect again to MySQL
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="securecheck"
)

# Use pandas to run SQL query
query = "SELECT * FROM traffic_logs LIMIT 5"
df_sql = pd.read_sql(query, con=mydb)

# Show result
df_sql.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_20292\4158437845.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, con=mydb)


,stop_datetime,country_name,driver_gender,driver_age,driver_race,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,2020-01-01 00:00:00,Canada,M,19,Asian,Speeding,1,Vehicle Search,Ticket,1,16-30 Min,1
1,2020-01-01 00:01:00,India,M,58,Other,Other,0,Vehicle Search,Arrest,1,16-30 Min,1
2,2020-01-01 00:02:00,USA,M,76,Black,Speeding,0,Frisk,Ticket,1,16-30 Min,1
3,2020-01-01 00:03:00,Canada,M,76,Black,DUI,1,Frisk,Warning,0,0-15 Min,1
4,2020-01-01 00:04:00,Canada,M,75,Other,Other,0,Vehicle Search,Arrest,1,16-30 Min,0
